## Notebook: Transform dữ liệu từ tầng broze lên tầng silver

### 1: Import & Connect DB
Cell này import thư viện cần thiết, load biến môi trường `.env` và tạo kết nối tới MySQL.  
- `bronze_engine` → đọc dữ liệu gốc từ schema Bronze.  
- `silver_engine` → ghi dữ liệu đã làm sạch sang schema Silver.  
Đây là bước khởi tạo, chưa có quyết định gì, chỉ thiết lập môi trường.


In [1]:
import os
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường
load_dotenv()

DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Kết nối DB
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")


### 2. Load dữ liệu từ Bronze
Ở đây chúng ta:
- Lấy toàn bộ bảng `customers_raw` từ Bronze.
- In ra số record (`Shape`).
- Hiển thị 1 record JSON thô để nắm cấu trúc.

📌 **Decision Point:** Từ JSON mẫu, ta quan sát để xác định có những nhóm trường nào (định danh, hành vi, địa chỉ...).


In [2]:
customers_df = pd.read_sql("SELECT * FROM customers_raw", bronze_engine)
print("Shape:", customers_df.shape)

# Hiển thị 1 record mẫu để nắm cấu trúc
print(json.dumps(json.loads(customers_df["raw_json"].iloc[0]), indent=2, ensure_ascii=False))


Shape: (36090, 4)
{
  "returned_order_count": 0,
  "is_block": false,
  "customer_id": "c7a583d6-392d-4308-a02b-67c001ee03db",
  "count_referrals": 0,
  "username": null,
  "emails": [],
  "inserted_at": "2025-08-16T02:19:59",
  "creator": null,
  "list_voucher": [],
  "name": "Thinh Bui",
  "assigned_user_id": null,
  "last_order_at": null,
  "date_of_birth": null,
  "id": "e906c5c7-ea19-42e5-a971-57cf09c94417",
  "fb_id": "377626778776391_31407011708897267",
  "notes": [],
  "total_amount_referred": null,
  "is_adjust_debts": null,
  "is_discount_by_level": true,
  "updated_at": "2025-08-16T02:19:59",
  "conversation_link": "https://pancake.vn/377626778776391?customer_id=c7a583d6-392d-4308-a02b-67c001ee03db",
  "order_count": 1,
  "order_sources": [
    "-1"
  ],
  "tags": [],
  "succeed_order_count": 0,
  "user_block_id": null,
  "phone_numbers": [
    "0903693389"
  ],
  "reward_point": 0,
  "shop_id": 230361475,
  "current_debts": 0,
  "level": null,
  "purchased_amount": 0,
  "cr

### 3: Liệt kê toàn bộ keys trong JSON
Cell này scan 200 record đầu tiên, lấy toàn bộ keys xuất hiện trong JSON.  
Kết quả cho ta danh sách ~40 keys của bảng customers.

📌 **Decision Point:** Từ danh sách này, ta quyết định chọn những cột quan trọng để parse (ví dụ: customer_id, name, gender, order_count...) và bỏ các cột ít giá trị (notes, creator...).


In [3]:
all_keys = set()
for raw in customers_df["raw_json"].head(200):   # scan 200 record đầu
    d = json.loads(raw)
    all_keys.update(d.keys())

print("===== Tổng số keys:", len(all_keys), " =====")
print(sorted(all_keys))

===== Tổng số keys: 40  =====
['active_levera_pay', 'assigned_user_id', 'conversation_link', 'conversation_tags', 'count_referrals', 'creator', 'creator_id', 'currency', 'current_debts', 'customer_id', 'date_of_birth', 'emails', 'fb_id', 'gender', 'id', 'inserted_at', 'is_adjust_debts', 'is_block', 'is_discount_by_level', 'last_order_at', 'level', 'list_voucher', 'name', 'notes', 'order_count', 'order_sources', 'phone_numbers', 'purchased_amount', 'referral_code', 'returned_order_count', 'reward_point', 'shop_customer_addresses', 'shop_id', 'succeed_order_count', 'tags', 'total_amount_referred', 'updated_at', 'used_reward_point', 'user_block_id', 'username']


### 4: Parse & Flatten JSON
Cell này định nghĩa hàm `parse_customer` để:
- Parse JSON từ trường `raw_json`.
- Flatten các trường nested như `phone_numbers`, `emails`, `order_sources`.
- Chỉ chọn những cột quan trọng cho phân tích (định danh, hành vi, trạng thái, địa chỉ).

Kết quả: DataFrame tabular (`customers_parsed`).

📌 **Decision Point:** Sau cell này, ta có bảng dạng sạch hơn để kiểm tra chất lượng dữ liệu.


In [4]:
def parse_customer(row):
    d = json.loads(row["raw_json"])
    addr = d.get("shop_customer_addresses")[0] if d.get("shop_customer_addresses") else {}

    return {
        # Định danh
        "customer_id": d.get("customer_id"),
        "shop_id": d.get("shop_id"),
        "name": d.get("name"),
        "gender": d.get("gender"),
        "date_of_birth": d.get("date_of_birth"),
        "phone": ",".join(d.get("phone_numbers", [])) if d.get("phone_numbers") else None,
        "email": ",".join(d.get("emails", [])) if d.get("emails") else None,
        "fb_id": d.get("fb_id"),

        # Hành vi mua hàng
        "order_count": d.get("order_count"),
        "succeed_order_count": d.get("succeed_order_count"),
        "returned_order_count": d.get("returned_order_count"),
        "purchased_amount": d.get("purchased_amount"),
        "reward_point": d.get("reward_point"),
        "used_reward_point": d.get("used_reward_point"),
        "current_debts": d.get("current_debts"),
        "count_referrals": d.get("count_referrals"),
        "total_amount_referred": d.get("total_amount_referred"),

        # Thời gian & trạng thái
        "inserted_at": d.get("inserted_at"),
        "updated_at": d.get("updated_at"),
        "last_order_at": d.get("last_order_at"),
        "is_block": d.get("is_block"),
        "is_discount_by_level": d.get("is_discount_by_level"),
        "active_levera_pay": d.get("active_levera_pay"),

        # Địa chỉ
        "province_id": addr.get("province_id"),
        "full_address": addr.get("full_address"),

        # Khác
        "referral_code": d.get("referral_code"),
        "order_sources": ",".join(d.get("order_sources", [])) if d.get("order_sources") else None,
        "tags": ",".join(d.get("tags", [])) if d.get("tags") else None,
    }

customers_parsed = customers_df.apply(parse_customer, axis=1, result_type="expand")
customers_parsed.head()


,customer_id,shop_id,name,gender,date_of_birth,phone,email,fb_id,order_count,succeed_order_count,...,updated_at,last_order_at,is_block,is_discount_by_level,active_levera_pay,province_id,full_address,referral_code,order_sources,tags
0,c7a583d6-392d-4308-a02b-67c001ee03db,230361475,Thinh Bui,male,None,0903693389,None,377626778776391_31407011708897267,1,0,...,2025-08-16T02:19:59,None,False,True,False,None,None,GM5wdxsy,-1,None
1,590ba7de-e4fe-43b3-8013-27fc69937edf,230361475,Truong Minh,male,None,0907809070,None,377626778776391_24536870619280831,1,0,...,2025-08-15T12:56:50,None,False,True,False,None,None,6zqy77Bu,-1,None
2,a9e62388-57c2-4834-a87c-7c1025aea92d,230361475,Quach Quach,male,None,0986533988,None,377626778776391_24712861355019072,1,0,...,2025-08-15T12:45:17,None,False,True,False,None,None,uyMB3h6E,-1,None
3,002810c4-5e96-4882-b85a-a85122e47108,230361475,Dung Vu,male,None,0977752936,None,377626778776391_24254873767534636,1,0,...,2025-08-15T12:04:38,None,False,True,False,None,None,SnU6T9jj,-1,None
4,cd25d874-d782-40d1-8561-5507bbd6114d,230361475,Phạm Mụi,male,None,0774443833,None,377626778776391_30782170181397391,1,0,...,2025-08-15T10:26:23,None,False,True,False,None,None,ng7CfaOM,-1,None


### 5. Data Quality Check
Mục tiêu:
- Tính tỷ lệ null từng cột → quyết định giữ/lược bỏ.
- Check duplicate theo `customer_id`.
- In sample values cho các field quan trọng (`gender`, `is_block`, `is_discount_by_level`, `active_levera_pay`).

📌 **Decision Point:** 
- Nếu cột null > 70% → cân nhắc drop.  
- Nếu có duplicate → xác định rule xử lý (giữ bản mới nhất).  
- Nếu có giá trị bất thường (vd: gender = "phi nhị giới") → quyết định chuẩn hóa mapping.

In [5]:
print("===== Tỉ lệ null =====")
null_ratio = customers_parsed.isnull().mean().sort_values(ascending=False)
display(null_ratio)

print("===== Duplicate check (customer_id) =====")
dup_count = customers_parsed["customer_id"].duplicated().sum()
print(f"Số lượng customer_id trùng: {dup_count}")

print("===== Sample values =====")
for col in ["gender", "is_block", "is_discount_by_level", "active_levera_pay"]:
    print(f"\n--- {col} ---")
    print(customers_parsed[col].value_counts(dropna=False).head(10))
    

===== Tỉ lệ null =====


email                    1.000000
tags                     1.000000
total_amount_referred    1.000000
used_reward_point        1.000000
date_of_birth            0.999501
last_order_at            0.260848
gender                   0.148545
province_id              0.100887
full_address             0.100610
fb_id                    0.003741
phone                    0.002494
order_sources            0.000831
name                     0.000194
returned_order_count     0.000000
purchased_amount         0.000000
order_count              0.000000
shop_id                  0.000000
customer_id              0.000000
succeed_order_count      0.000000
reward_point             0.000000
updated_at               0.000000
inserted_at              0.000000
current_debts            0.000000
count_referrals          0.000000
active_levera_pay        0.000000
is_discount_by_level     0.000000
is_block                 0.000000
referral_code            0.000000
dtype: float64

===== Duplicate check (customer_id) =====
Số lượng customer_id trùng: 0
===== Sample values =====

--- gender ---
gender
female          19486
male            11239
None             5361
Phi nhị giới        3
Nonbinary           1
Name: count, dtype: int64

--- is_block ---
is_block
False    36089
True         1
Name: count, dtype: int64

--- is_discount_by_level ---
is_discount_by_level
True    36090
Name: count, dtype: int64

--- active_levera_pay ---
active_levera_pay
False    36087
True         3
Name: count, dtype: int64


📌 Nhận xét

- Các cột như shop_id, is_discount_by_level, is_block, active_levera_pay có tỷ lệ lặp lại gần 100% → hầu như constant, không mang nhiều giá trị phân tích.

- Các cột email, tags, total_amount_referred, used_reward_point cũng vừa null rất nhiều, vừa lặp gần 100% → loại bỏ là hợp lý.

- gender có tỷ lệ lặp lại cao vì đa số khách là male/female, nhưng vẫn hữu ích cho phân tích phân khúc → nên giữ.

- reward_point, current_debts, count_referrals gần như luôn = 0 → nếu business hiện chưa dùng thì có thể tạm bỏ, nhưng nếu muốn track loyalty/credit thì nên giữ.

### 6. Làm sạch dữ liệu
Trong cell này:
- Viết hàm `drop_high_null_cols` để tự động drop cột null > 70%.
- Chuẩn hóa datetime (inserted_at, updated_at, last_order_at...).
- Ép kiểu numeric cho các cột số.
- Chuẩn hóa `gender` về M/F/O.
- Kết quả: DataFrame `customers_clean` đã sẵn sàng để load vào Silver.

📌 **Decision Point:** In ra danh sách cột bị drop. Từ đó, ta quyết định có cần override giữ lại cột nào (ví dụ tags, date_of_birth) dù null cao không.


In [6]:

def drop_high_null_cols(df, threshold=0.7):
    """
    Xóa các cột có tỷ lệ null > threshold (mặc định 70%).
    """
    null_ratio = df.isnull().mean()
    cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()
    
    print(f"⚠️ Các cột bị drop (null > {threshold*100}%): {cols_to_drop}")
    
    return df.drop(columns=cols_to_drop, errors="ignore")

# Drop cột null > 70%
customers_clean = drop_high_null_cols(customers_parsed, threshold=0.7)

# Chuẩn hóa datetime (chỉ convert nếu cột tồn tại)
datetime_cols = ["inserted_at", "updated_at", "last_order_at", "date_of_birth"]
for col in datetime_cols:
    if col in customers_clean.columns:
        customers_clean[col] = pd.to_datetime(customers_clean[col], errors="coerce")

# Ép kiểu numeric
num_cols = [
    "order_count", "succeed_order_count", "returned_order_count",
    "purchased_amount", "reward_point", "used_reward_point",
    "current_debts", "count_referrals", "total_amount_referred"
]
for col in num_cols:
    if col in customers_clean.columns:
        customers_clean[col] = pd.to_numeric(customers_clean[col], errors="coerce")

# Chuẩn hóa gender
if "gender" in customers_clean.columns:
    customers_clean["gender"] = customers_clean["gender"].str.lower().replace({
        "male": "M", "female": "F",
        "nam": "M", "nữ": "F",
        "phi nhị giới": "O", "nonbinary": "O"
    })

customers_clean.head()


⚠️ Các cột bị drop (null > 70.0%): ['date_of_birth', 'email', 'used_reward_point', 'total_amount_referred', 'tags']


,customer_id,shop_id,name,gender,phone,fb_id,order_count,succeed_order_count,returned_order_count,purchased_amount,...,inserted_at,updated_at,last_order_at,is_block,is_discount_by_level,active_levera_pay,province_id,full_address,referral_code,order_sources
0,c7a583d6-392d-4308-a02b-67c001ee03db,230361475,Thinh Bui,M,0903693389,377626778776391_31407011708897267,1,0,0,0,...,2025-08-16 02:19:59,2025-08-16 02:19:59,NaT,False,True,False,None,None,GM5wdxsy,-1
1,590ba7de-e4fe-43b3-8013-27fc69937edf,230361475,Truong Minh,M,0907809070,377626778776391_24536870619280831,1,0,0,0,...,2025-08-15 12:56:50,2025-08-15 12:56:50,NaT,False,True,False,None,None,6zqy77Bu,-1
2,a9e62388-57c2-4834-a87c-7c1025aea92d,230361475,Quach Quach,M,0986533988,377626778776391_24712861355019072,1,0,0,0,...,2025-08-15 12:45:17,2025-08-15 12:45:17,NaT,False,True,False,None,None,uyMB3h6E,-1
3,002810c4-5e96-4882-b85a-a85122e47108,230361475,Dung Vu,M,0977752936,377626778776391_24254873767534636,1,0,0,0,...,2025-08-15 12:04:38,2025-08-15 12:04:38,NaT,False,True,False,None,None,SnU6T9jj,-1
4,cd25d874-d782-40d1-8561-5507bbd6114d,230361475,Phạm Mụi,M,0774443833,377626778776391_30782170181397391,1,0,0,0,...,2025-08-15 10:26:23,2025-08-15 10:26:23,NaT,False,True,False,None,None,ng7CfaOM,-1


### 7. Load vào Silver
- Sử dụng `to_sql` để tạo bảng `dim_customers`.
- `if_exists="replace"` → ghi đè dữ liệu cũ. Khi chạy production có thể đổi sang `append` để incremental load.
- Dữ liệu lúc này đã sạch, có thể join được với các bảng dimension/fact khác trong Silver.

📌 **Decision Point:** 
- Xác nhận số record load thành công.
- Kiểm tra schema ở Silver có đúng với kỳ vọng (kiểu dữ liệu, số cột, số dòng).

In [7]:
table_name = "dim_customers"

customers_clean.to_sql(
    table_name,
    con=silver_engine,
    if_exists="replace",   # hoặc 'append' nếu incremental load
    index=False,
)

print(f"✅ Đã load {customers_clean.shape[0]} records vào Silver: {table_name}")


✅ Đã load 36090 records vào Silver: dim_customers


### 8. Sinh Data Dictionary
**Data Dictionary** để mô tả cấu trúc bảng Silver:

- `column` → tên cột.
- `dtype` → kiểu dữ liệu (theo pandas, có thể map sang SQL).
- `null_pct` → tỷ lệ null.
- `nunique` → số lượng giá trị duy nhất.
- `sample_value` → ví dụ 1 giá trị trong cột.

Ý nghĩa:
- Giúp hiểu rõ hơn chất lượng dữ liệu.
- Là tài liệu tham chiếu khi thiết kế Star Schema ở Gold.
- Là bằng chứng cho Data Governance/Data Quality.

📌 **Decision Point:** 
- Dựa vào Data Dictionary, kiểm tra lại schema Silver có phù hợp với Business Requirement.
- Nếu thiếu cột quan trọng hoặc dtype chưa chuẩn, cần quay lại bước 4–6 để chỉnh sửa.


In [8]:
data_dict = pd.DataFrame({
    "columns": customers_clean.columns,
    "dtype": customers_clean.dtypes.astype(str),
    "null_pct": customers_clean.isnull().mean().round(3).values,
    "nunique": customers_clean.nunique().values,
    "sample_value": customers_clean.astype(str).iloc[0].values
})

print("===== Data Dictionary (Customers Clean) =====")
data_dict


===== Data Dictionary (Customers Clean) =====


,columns,dtype,null_pct,nunique,sample_value
customer_id,customer_id,object,0.000,36090,c7a583d6-392d-4308-a02b-67c001ee03db
shop_id,shop_id,int64,0.000,1,230361475
name,name,object,0.000,25253,Thinh Bui
gender,gender,object,0.149,3,M
phone,phone,object,0.002,35716,0903693389
fb_id,fb_id,object,0.004,35955,377626778776391_31407011708897267
order_count,order_count,int64,0.000,11,1
succeed_order_count,succeed_order_count,int64,0.000,6,0
returned_order_count,returned_order_count,int64,0.000,5,0
purchased_amount,purchased_amount,int64,0.000,538,0
